# 빅프9조

분석방법
순찰 영역은 원형이지만 격자점 내부에서는 분석을 정사각형으로 함

# 기본 설정

 필수 라이브러리 import

In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

import folium
from IPython.display import display

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 아래와 같이 경찰서 관련 데이터는 {경찰서이름 : 데이터} 형식으로 관리

파일 관련 상수 설정

In [2]:
data_path = "전처리완료데이터/"
encoding = "cp949"
extension = ".csv"

경찰서 관련 정보 설정

In [3]:
police = pd.read_csv(data_path + "police.csv", encoding=encoding)
police = {police.loc[i, "이름"]: police.loc[i, ["위도", "경도"]].values.astype(np.float64) for i in police.index}

police_path     = {k: f'전처리완료데이터/police/{k}/' for k in police.keys()}
police_raw_path = {k: f'전처리필요데이터/police/{k}/' for k in police.keys()}

police

{'당현지구대': array([ 37.64201519, 127.0683536 ]),
 '노원역지구대': array([ 37.65500523, 127.0669163 ]),
 '화랑지구대': array([ 37.62256592, 127.0745976 ]),
 '불암지구대': array([ 37.65243504, 127.0773404 ]),
 '월계지구대': array([ 37.63368623, 127.0592911 ]),
 '마들지구대': array([ 37.66458702, 127.0636189 ]),
 '당고개파출소': array([ 37.67147105, 127.0778527 ]),
 '상계1파출소': array([ 37.67968126, 127.0550472 ])}

격자점 생성에 필요한 상수 선언

In [4]:
NS_DIFF_1M = 0.000007
EW_DIFF_1M = 0.000009
PATROL_HR = 900
GRID_DIFF = 90

### 데이터를 순찰 반경 안의 것만 잘라내는 함수

In [5]:
def filtering(data, police, mode = "시설"):
    distance = PATROL_HR + GRID_DIFF/2 + 5 if mode == "시설" else PATROL_HR + 10 
    ft = (data[["위도", "경도"]] - police).copy()
    ft.loc[:, "위도"] /= NS_DIFF_1M
    ft.loc[:, "경도"] /= EW_DIFF_1M
    ft = np.sqrt( np.sum( np.square(ft), axis=1 ))
    return data.loc[ft <= distance, :].reset_index(drop=True)

# 기본 격자점 생성

기본 격자점 반환함수

In [6]:
def grids(police):
    hcount = PATROL_HR//GRID_DIFF
    ew = np.arange(-hcount, hcount + 1, dtype=np.float64)
    ns = ew.reshape(-1,1).copy()
    ew *= EW_DIFF_1M * GRID_DIFF
    ns *= NS_DIFF_1M * GRID_DIFF
    
    result = np.full((2*hcount+1, 2*hcount+1, 2), police, dtype=np.float64)
    result[:,:,0] += ns
    result[:,:,1] += ew
    
    return pd.DataFrame(result.reshape(-1,2), columns=["위도", "경도"])

원형 격자점 생성

In [7]:
police_grid = {k:grids(v) for k, v in police.items()}
for k,v in police_grid.items():
    police_grid[k] = filtering(v, police[k], k)
police_grid

{'당현지구대':             위도          경도
 0    37.635715  127.067544
 1    37.635715  127.068354
 2    37.635715  127.069164
 3    37.636345  127.065114
 4    37.636345  127.065924
 ..         ...         ...
 320  37.647685  127.070784
 321  37.647685  127.071594
 322  37.648315  127.067544
 323  37.648315  127.068354
 324  37.648315  127.069164
 
 [325 rows x 2 columns],
 '노원역지구대':             위도          경도
 0    37.648705  127.066106
 1    37.648705  127.066916
 2    37.648705  127.067726
 3    37.649335  127.063676
 4    37.649335  127.064486
 ..         ...         ...
 320  37.660675  127.069346
 321  37.660675  127.070156
 322  37.661305  127.066106
 323  37.661305  127.066916
 324  37.661305  127.067726
 
 [325 rows x 2 columns],
 '화랑지구대':             위도          경도
 0    37.616266  127.073788
 1    37.616266  127.074598
 2    37.616266  127.075408
 3    37.616896  127.071358
 4    37.616896  127.072168
 ..         ...         ...
 320  37.628236  127.077028
 321  37.628236  127.0

# 장소별 범죄 발생 확률 이용

### 샘플링

뽑을 개수 (5개)

In [8]:
SAMPLING = 5

격자점 범위 내에서 무작위로 5개 점 추가

In [9]:
def sampled_grids(grid):
    interval_size = SAMPLING+1
    result = np.zeros((grid.shape[0]*interval_size, 2), dtype=np.float64)
    for i in range(0, result.shape[0], interval_size):
        ran = np.zeros((interval_size, 2), dtype=np.float64)
        ran[1:] = np.random.uniform(-1, 1, (interval_size-1, 2))
        ran[:, 0] *= (GRID_DIFF/2)*NS_DIFF_1M
        ran[:, 1] *= (GRID_DIFF/2)*EW_DIFF_1M
        ran += grid.values[i//interval_size]
        result[i:i+interval_size] += ran
    return pd.DataFrame(result, columns = ["위도", "경도"])

추가한 데이터 csv내보내기

In [10]:
police_sampled = {k: sampled_grids(v) for k,v in police_grid.items()}
police_sampled

{'당현지구대':              위도          경도
 0     37.635715  127.067544
 1     37.635825  127.067748
 2     37.635839  127.067600
 3     37.635843  127.067191
 4     37.635852  127.067920
 ...         ...         ...
 1945  37.648148  127.069229
 1946  37.648452  127.069372
 1947  37.648151  127.069356
 1948  37.648366  127.069077
 1949  37.648205  127.068941
 
 [1950 rows x 2 columns],
 '노원역지구대':              위도          경도
 0     37.648705  127.066106
 1     37.648585  127.065747
 2     37.648555  127.066042
 3     37.648880  127.065827
 4     37.648714  127.066463
 ...         ...         ...
 1945  37.661319  127.067419
 1946  37.661081  127.068103
 1947  37.661292  127.068045
 1948  37.661556  127.068066
 1949  37.661119  127.067800
 
 [1950 rows x 2 columns],
 '화랑지구대':              위도          경도
 0     37.616266  127.073788
 1     37.616404  127.073761
 2     37.616482  127.074080
 3     37.616292  127.074179
 4     37.616220  127.073769
 ...         ...         ...
 1945  37.628693 

In [11]:
'''
for k,v in police_sampled.items():
    v.to_csv(raw_path + k + "_sampled_grid.csv", index=False, encoding=encoding)
'''

'\nfor k,v in police_sampled.items():\n    v.to_csv(raw_path + k + "_sampled_grid.csv", index=False, encoding=encoding)\n'

### 주소로 변환

https://www.geoservice.co.kr/

내보낸 csv 파일 이용, 위 링크에서 역지오코딩, 주소얻기

주소 기본 데이터 생성

In [12]:
def addr_data(file_path):
    data=pd.read_csv(file_path, encoding=encoding)
    data["이름"] = None
    data["확률"] = None
    data["index"]=[i//(SAMPLING+1) for i in range(data.shape[0])]
    return data

In [13]:
police_addr = {k : addr_data(f'{police_path[k]}{k}_sampled_addr.csv') for k in police.keys()}
police_addr

{'당현지구대':              위도          경도                 지번주소                 도로명주소    이름  \
 0     37.635715  127.067544  서울특별시 노원구 하계동 270-3  서울특별시 노원구 한글비석로 47-2  None   
 1     37.635727  127.067823  서울특별시 노원구 하계동 270-3  서울특별시 노원구 한글비석로 47-2  None   
 2     37.635488  127.067734  서울특별시 노원구 하계동 270-3  서울특별시 노원구 한글비석로 46-2  None   
 3     37.635899  127.067738    서울특별시 노원구 하계동 270     서울특별시 노원구 섬밭로 232  None   
 4     37.635524  127.067902    서울특별시 노원구 하계동 273  서울특별시 노원구 한글비석로 46-2  None   
 ...         ...         ...                  ...                   ...   ...   
 1945  37.648151  127.069158  서울특별시 노원구 중계동 514-3     서울특별시 노원구 노원로 331  None   
 1946  37.648573  127.068931  서울특별시 노원구 중계동 514-6     서울특별시 노원구 노원로 331  None   
 1947  37.648504  127.069279  서울특별시 노원구 중계동 514-3     서울특별시 노원구 노원로 331  None   
 1948  37.648451  127.068973  서울특별시 노원구 중계동 514-6     서울특별시 노원구 노원로 331  None   
 1949  37.648320  127.069495  서울특별시 노원구 중계동 514-4     서울특별시 노원구 노원로 331  None   
 
         확률  inde

# 건물 이름 크롤링

지연 시간(1초), 웹 지도 url(카카오맵) 설정

In [14]:
DELAY_TIME = 0.5
map_url = "https://map.kakao.com/"

브라우저 드라이버 반환(no 전달시 창 안뜸)

In [15]:
def openweb(mode="window"):
    if mode == "no":
        options = wb.ChromeOptions()
        options.add_argument('headless')
        driver = wb.Chrome(options=options)
    else:
        driver = wb.Chrome()
    driver.get(map_url)
    time.sleep(DELAY_TIME)
    return driver

검색어 입력 후 실행

In [16]:
def search(browser, address):
    search_box = browser.find_element(By.ID, "search.keyword.query")
    while search_box.get_attribute("value"):
        search_box.send_keys(Keys.CONTROL, 'a')
        search_box.send_keys(Keys.BACKSPACE)
    search_box.send_keys(address)
    search_box.send_keys(Keys.RETURN)
    time.sleep(DELAY_TIME)

### 크롤링 함수

도로나 강 야외의 경우 지번주소는 있지만 우편번호가 없다

지번주소 먼저 검색 -> 야외 분류

우편번호 있으면 -> 도로명주소 검색

건물이름 뜨면 -> 이름 반환

안뜨면 -> 우편번호 반환

In [17]:
def find_name(browser, addr, road_addr):
    if "산" in addr:
        return "산"
    if "지하" in road_addr:
        return "지하철"
    search(browser, addr)
    try:
        postage = browser.find_element(By.CLASS_NAME, "zip")
    except NoSuchElementException:
        return "nozip"
    else:
        postnum = postage.text
        search(browser, road_addr)
        try:
            buildname = browser.find_element(By.CLASS_NAME, "building")
        except NoSuchElementException:
            return postnum
        else:
            return buildname.text

주소 전부 검색 후 csv로 내보내기

In [18]:
def search_to_csv(browser, data, file_name):
    for i in range(len(data.index)):
        data.iloc[i, 4] = find_name(browser, data.iloc[i, 2], data.iloc[i, 3])
    data.to_csv(file_name, index=False, encoding=encoding)

In [ ]:
for k, v in police_addr.items():
    browser = openweb("no")
    search_to_csv(browser, v, f'{police_path[k]}{k}_sampled_name.csv')
    browser.quit()

### 이름을 토대로 유형 분류, 확률 점수 부여

In [ ]:
police_per = {k:pd.read_csv(data_path + k + "_sampled_name.csv", encoding=encoding) for k in police.keys()}
for k,v in police_per.items():
    print(k)
    set(v["이름"])

장소 분류에 따른 범죄확률

In [ ]:
crime_per = pd.read_csv(data_path+"processed_crime.csv", encoding=encoding)
crime_per

분류 결과를 바탕으로 이름이 어떤유형인지 분류

In [ ]:
def namespace(file_path):
    space = dict()
    with open(file_path, 'r', encoding='UTF8') as file:
        for i in file:
            temp=i.split(" - ")
            space[temp[0]] = temp[1].strip()
    return space

자료에 맞춰 확률 계산후 대입

In [ ]:
def insert_percentage(text_file_path, addr_data):
    space = namespace(text_file_path)
    for i in space:
        addr_data.loc[addr_data["이름"]==i, "확률"] = crime_per.loc[0, space[i]]

In [ ]:
for k, v in police_per.items():
    insert_percentage(data_path + k + "_name_category.txt", v)       
police_per

동일 지역에서 샘플된 데이터끼리 확률 합 구하고 격자점 데이터에 새로운 열로 추가

In [ ]:
def insert_location(grid, percentage_data):
    location = percentage_data[["index", "확률"]].groupby("index").sum()
    grid["location"] = location

In [ ]:
for k in police.keys():
    insert_location(police_grid[k], police_per[k])
police_grid

# 시설물 데이터 이용

시설물 점수 기준

시설물 밀도 계산 => 격자점 반경 내 개수 / 전체 범위 내 개수

이때 데이터를 순찰반경에 맞게 필터링해 쓸 것이므로 전체 범위 개수는 필터링된 데이터프레임의 행 개수이다

긍정적 시설(cctv 등)일 경우 밀도 뺌

부정적 시설(유흥업소 등)일 경우 밀도 더함

### 데이터 준비

안심시설물, cctv, 보안등 유흥업소

전체데이터는 너무 많으므로 filtering() 이용

긍정요소, 부정요소 정의

In [ ]:
def set_data(police):
    failcity_data = {"ansim" : pd.read_csv(data_path+"processed_ansim.csv", encoding=encoding),
                     "cctv"  : pd.read_csv(data_path+"processed_cctv.csv", encoding=encoding),
                     "light" : pd.read_csv(data_path+"processed_light.csv", encoding=encoding),
                     "salon" : pd.read_csv(data_path+"processed_salon.csv", encoding=encoding),}
    for k, v in failcity_data.items():
        failcity_data[k]=filtering(v,police)
    return failcity_data
    
positive = ["ansim", "cctv", "light"]
negative = ["salon"]

해당 범위 안에 목표물이 몇개 있는가?

In [ ]:
def location_count(data, location):
    ns_min = location.iloc[0] - GRID_DIFF/2*NS_DIFF_1M
    ns_max = location.iloc[0] + GRID_DIFF/2*NS_DIFF_1M
    ew_min = location.iloc[1] - GRID_DIFF/2*EW_DIFF_1M
    ew_max = location.iloc[1] + GRID_DIFF/2*EW_DIFF_1M
    return len(data.loc[(data["위도"] >= ns_min)&
                         (data["위도"] <= ns_max)&
                         (data["경도"] >= ew_min)&
                         (data["경도"] <= ew_max) , :].index)

격자점 데이터에 시설 밀도 열 추가하는 함수

In [ ]:
def insert_facility(grid, data):
    for k, v in data.items():
        for i in grid.index:
            grid.loc[i, k] = location_count(v, grid.loc[i, ["위도", "경도"]])
        total = grid.loc[:, k].sum()
        if total != 0:
            grid.loc[:, k] /= grid.loc[:, k].sum()
        else:
            grid.loc[:, k] = 0

밀도를 토대로 점수 계산

In [ ]:
def score(grid):
    grid["score"] = grid["location"]
    grid.drop(columns=["location"], inplace=True)
    for k in failcity_data.keys():
        mode = 1 if k in positive else -1
        grid["score"] += mode*grid[k]
        grid.drop(columns=[k], inplace=True)

밀도 계산후 열로 추가

In [ ]:
for k in police.keys():
    insert_facility(police_grid[k], set_data(police[k]))
police_grid

점수 계산후 위경도, 점수 열만 남기기

In [ ]:
for v in police_grid.values():
    score(v)
police_grid

# 지도에 시각화

지도에 해당 위도경도에 위치한 점 찍기

In [ ]:
def draw_marker(m, grid, color):
    color = "#" + color
    for i in grid[["위도","경도"]].values:
        folium.Circle(location=i, radius=2, color=color, fill_color=color).add_to(m)

시각화를 위해 점수를 -255(초록)~255(빨강)으로 변환

변환후 rgb코드에 맞게 문자열 생성 안전->초록, 위험->빨강

In [ ]:
def colorstr(num):
    b = num>=0
    num=hex(int(255-abs(num)))[2:]
    if len(num)==1:
        num="0"+num
    return "#ff"+num*2 if b else "#"+num+"ff"+num

In [ ]:
def score_to_color(grid):
    result = grid["score"].copy()
    mins = result.min()
    maxs = result.max()
    result = ((result - mins) / (maxs - mins) * 2 - 1) * 255
    result = result.apply(colorstr)
    return result

격자점 점수 시각화

In [ ]:
def draw_score(m, grid, color):
    for i in grid.index:
        folium.Circle(location=grid.iloc[i,:2].values,
                      radius=GRID_DIFF/2,
                      color=color[i],
                      fill=True,
                      fill_color=color[i]).add_to(m)

지도객체 생성 , 이름 붙이기

In [ ]:
m = {k: folium.Map(location = v, zoom_start=60) for k,v in police.items()}

for k, v in m.items():
    title_html = f'<h3 align="center" style="font-size:30px"><b>{k}</b></h3>'
    v.get_root().html.add_child(folium.Element(title_html))

경찰서, 격자점, 모든 시설물 표시

In [ ]:
for k, v in m.items():
    folium.Marker(location=police[k], 
                  popup=k,
                  icon=folium.Icon(color='lightblue', icon='star') ).add_to(v)

failcity_data_color = {"ansim" : '0000ff',
                       "cctv"  : '00ffff',
                       "light" : 'ffff00',
                       "salon" : 'ff00ff'}

for k in police.keys():
    draw_marker(m[k], police_grid[k], '000000')
    
for pk, pv in police.items():
    for fk, fv in set_data(pv):
        draw_marker(m[k], fv, failcity_data_color[fk])

점수 시각화

In [ ]:
for k in police.keys():
    draw_score(m[k],
               police_grid[k],
               score_to_color(police_grid[k]))
    display(m[k])